In [2]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
import logging
from functools import partial
from copy import deepcopy
from toolz import compose, valmap, keyfilter, identity, merge
from itertools import combinations
import pickle

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import torch

import sbi
import sbi.utils
import sbi.inference
import sbi.analysis

import sbibm
import sbibm.tasks.eggbox
import sbibm.tasks.eggbox.posterior
import sbibm.tasks.rotated_eggbox.posterior as REP

import scipy
import scipy.stats

import swyft
import swyft.utils.constrainedcorner as cc

import tmnre
import tmnre.metrics
import tmnre.benchmark
import tmnre.coverage.oned
from tmnre.nn.resnet import make_resenet_tail
from tmnre.marginalize import filter_marginals_by_dim

# Let's create a simulator in 2d

If $x = g(\theta)$ is the usual case, and we want our posterior to be $p(A \theta | x)$, we need to rotate such that $x = g(A^{-1} \theta)$.


## Actually...
This is a waste of time. We can just use our posterior samples from $x=g(\theta)$. We rotate them with $\theta \to A \theta$.

In [52]:
def rot(angle: float, rdim1: int, rdim2: int, dim: int):
    assert rdim1 != rdim2
    r = np.eye(dim)
    r[rdim1, rdim1] = np.cos(angle)
    r[rdim1, rdim2] = -np.sin(angle)
    r[rdim2, rdim1] = np.sin(angle)
    r[rdim2, rdim2] = np.cos(angle)
    return r

rot2 = partial(rot, rdim1=0, rdim2=1, dim=2)
fortyfive = rot2(np.pi/4)
invfortyfive = np.linalg.inv(fortyfive)
print(fortyfive)

def g(theta):
    return np.sin(np.pi * invfortyfive @ theta) ** 2

dim = 2
theta0_before = np.ones(dim) * 0.25
theta0 = fortyfive @ theta0_before
x0 = g(theta0)

normal = scipy.stats.multivariate_normal(mean=x0, cov=0.1 * np.eye(dim))
loglikelihood = compose(normal.logpdf, g)

[[ 0.70710678 -0.70710678]
 [ 0.70710678  0.70710678]]


In [58]:
bounds = np.array(
    [[0., 0.],
     [1., 0.],
     [0., 1.],
     [1., 1.]]
)

fortyfive @ np.ones(dim)

array([1.11022302e-16, 1.41421356e+00])

# manually producing the rotation matrix s.t. 1s vector maps to [1, 0, ...]

In [92]:
aaa = np.block([[fortyfive, np.zeros_like(fortyfive)], [np.zeros_like(fortyfive), fortyfive]])
print(aaa @ np.ones(dim * 2))

rot(np.pi / 4, 1, 3, 4) @ aaa @ np.ones(dim * 2)

[1.11022302e-16 1.41421356e+00 1.11022302e-16 1.41421356e+00]


array([1.11022302e-16, 1.11022302e-16, 1.11022302e-16, 2.00000000e+00])

In [118]:
rij = partial(rot, angle=np.pi/4, dim=6)
aaa = rij(rdim1=0, rdim2=1) @ rij(rdim1=2, rdim2=3) @ rij(rdim1=4, rdim2=5)
print(aaa @ np.ones(6))
aaa = rij(rdim1=3, rdim2=5) @ rij(rdim1=1, rdim2=3) @ aaa
print(aaa @ np.ones(6))

[1.11022302e-16 1.41421356e+00 1.11022302e-16 1.41421356e+00
 1.11022302e-16 1.41421356e+00]
[1.11022302e-16 1.11022302e-16 1.11022302e-16 4.14213562e-01
 1.11022302e-16 2.41421356e+00]


In [122]:
d = 8
rij = partial(rot, angle=np.pi/4, dim=d)
aaa = rij(rdim1=0, rdim2=1) @ rij(rdim1=2, rdim2=3) @ rij(rdim1=4, rdim2=5) @ rij(rdim1=6, rdim2=7)
print(aaa @ np.ones(d))


# todo
aaa = rij(rdim1=2, rdim2=3) @ rij(rdim1=4, rdim2=5) @ rij(rdim1=6, rdim2=7)


# aaa = rij(rdim1=3, rdim2=5) @ rij(rdim1=1, rdim2=3) @ aaa
# print(aaa @ np.ones(6))

[1.11022302e-16 1.41421356e+00 1.11022302e-16 1.41421356e+00
 1.11022302e-16 1.41421356e+00 1.11022302e-16 1.41421356e+00]


In [ ]:
rij = partial(rot, angle=np.pi/4, dim=6)
aaa = rij(rdim1=0, rdim2=1) @ rij(rdim1=2, rdim2=3) @ rij(rdim1=4, rdim2=5)
print(aaa @ np.ones(6))
aaa = rij(rdim1=1, rdim2=3) @ aaa
aaa = rij(rdim1=3, rdim2=5) @ aaa
print(aaa @ np.ones(6))

In [116]:
d = 6

out = []
for i in range(0, d // 2):
    out.append((i, i+1))
for j in range(len(out) - 1):
    out.append((2 * j + 1, 2 * j + 3))
print(out)

[(0, 1), (1, 2), (2, 3), (1, 3), (3, 5)]


4

In [62]:
fortyfive @ bounds.T

array([[ 0.00000000e+00,  7.07106781e-01, -7.07106781e-01,
         1.11022302e-16],
       [ 0.00000000e+00,  7.07106781e-01,  7.07106781e-01,
         1.41421356e+00]])

In [31]:
def project_SOn(a):
    r0 = scipy.linalg.fractional_matrix_power(a.T @ a, 1/2)
    r1 = np.linalg.inv(r0)
    return a @ r1

from scipy.spatial.transform import Rotation as R

In [39]:
dim = 3
x = compose(np.tril, np.ones)((dim, dim)) * 2 - 1
R.from_euler("ZYZ", [np.pi/4, np.pi/4, 0], ).as_matrix() @ x

array([[ 0.29289322, -0.70710678,  0.70710678],
       [ 1.70710678,  0.70710678, -0.70710678],
       [ 0.        ,  1.41421356,  1.41421356]])

In [35]:
np.round(R.from_euler("zyz", [np.pi/4, np.arctan(1/np.sqrt(2)), 0], ).as_matrix() @ v, 2)

array([[ 0.58, -1.73],
       [ 1.41, -0.  ],
       [ 0.82,  0.  ]])

In [34]:
v = np.asarray(
    [[1., 1., 1.,],
     [-1., 1., -1.],
    ]).T


array([[ 0.58, -1.73],
       [ 1.41, -0.  ],
       [ 0.82,  0.  ]])

In [ ]:
sigma = 0.1

def get_indicator_function(lower_x: float, upper_x: float, fn = identity):
    return lambda xx: np.where((lower_x <= xx) & (xx <= upper_x), fn(xx), np.zeros_like(xx))

def g(theta): 
    return np.sin(np.pi * theta) ** 2

def get_unnormalized_posterior_pdf(x: float, sigma: float = 0.1) -> np.ndarray:
    normal_pdf = compose(np.exp, scipy.stats.norm(loc=x, scale=sigma).logpdf)
    return get_indicator_function(
        -0.5, 0.5, lambda theta: compose(normal_pdf, g)(theta)
    )

post_pdf = get_unnormalized_posterior_pdf(0.5)
x = np.linspace(-0.5, 0.5, 1000)
plt.plot(x, post_pdf(x))

# todo I think I can just generate samples in this configuration then transport them to where I want them

In [28]:
dim = 3

# set initial postitions
x = compose(np.tril, np.ones)((dim, dim)) * 2 - 1
# print(x)
# x = x / np.linalg.norm(x, axis=1)

# set rotated positions
b = x.copy()
b[:, 0] = np.linalg.norm(x[:, 0]) * np.asarray([1.] + [0.] * (dim - 1))
# b = np.eye(dim) * np.linalg.norm(x, axis=1)[0]

# determine rotation matrix
a = b @ np.linalg.inv(x)

# print(np.linalg.det(a))
# np.round(a)
# np.round(a, 5) @ x
a @ x

array([[ 1.73205081, -1.        , -1.        ],
       [ 0.        ,  1.        , -1.        ],
       [ 0.        ,  1.        ,  1.        ]])

In [30]:
project_SOn(a) @ x

array([[ 1.53503135, -0.23747018, -0.79568075],
       [ 0.48342454,  0.84461565, -1.04163511],
       [ 0.6402964 ,  1.49339624,  1.13220512]])

In [15]:
np.linalg.matrix_power(a.T @ a, 1/2)

TypeError: exponent must be an integer

In [16]:
np.linalg?

Type:        module
String form: <module 'numpy.linalg' from '/home/ben/sci/tmnre/envs/lib/python3.8/site-packages/numpy/linalg/__init__.py'>
File:        ~/sci/tmnre/envs/lib/python3.8/site-packages/numpy/linalg/__init__.py
Docstring:  
``numpy.linalg``

The NumPy linear algebra functions rely on BLAS and LAPACK to provide efficient
low level implementations of standard linear algebra algorithms. Those
libraries may be provided by NumPy itself using C versions of a subset of their
reference implementations but, when possible, highly optimized libraries that
take advantage of specialized processor functionality are preferred. Examples
of such libraries are OpenBLAS, MKL (TM), and ATLAS. Because those libraries
are multithreaded and processor dependent, environmental variables and external
packages such as threadpoolctl may be needed to control the number of threads
or specify the processor architecture.

- OpenBLAS: https://www.openblas.net/
- threadpoolctl: https://github.com/joblib/t

In [18]:
np.linalg.eig(a.T @ a)

(array([2.72474487, 1.        , 0.27525513]),
 array([[ 0.89127898, -0.4472136 , -0.07497853],
        [-0.34402085, -0.77459667,  0.53070675],
        [ 0.2954174 ,  0.4472136 ,  0.84423253]]))

0.9999999999999993